# Предобработка данных для загрузки в ХД

Наши данные достаточно сложные и "грязные", поэтому было принято решение предобработать их с помощью Python. Через данную процедуру можно "пропустить" любой файл. Формат входного файла легко меняется.

In [1]:
import pandas as pd
import numpy as np
import operator
from tqdm import tqdm

# Загрузка файла

In [2]:
data = pd.read_excel('kofe-2.xlsx')
data.drop('id', axis=1, inplace=True)
data.shape

(1354, 52)

In [3]:
data.describe()

,view_certificate_2,Cupping Protocol and Descriptors,View Green Analysis Details,Request a Sample,Number of Bags,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean Cup,Sweetness,Cupper Points,NA.2,Moisture,Quakers,NA.3,Unnamed: 51
count,0.0,0.0,0.0,0.0,1340.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,0.0,1338.000000,1337.000000,0.0,0.0
mean,NaN,NaN,NaN,NaN,154.067910,7.565822,7.519447,7.400321,7.535456,7.519925,7.516114,9.827780,9.828012,9.846809,7.502571,NaN,0.088281,0.173523,NaN,NaN
std,NaN,NaN,NaN,NaN,130.006861,0.380020,0.400681,0.405779,0.380310,0.361393,0.417102,0.616347,0.810022,0.684383,0.475218,NaN,0.048359,0.832419,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,14.000000,7.420000,7.330000,7.250000,7.330000,7.330000,7.330000,10.000000,10.000000,10.000000,7.250000,NaN,0.090000,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,172.500000,7.580000,7.580000,7.420000,7.580000,7.500000,7.500000,10.000000,10.000000,10.000000,7.500000,NaN,0.110000,0.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,275.000000,7.750000,7.750000,7.580000,7.750000,7.670000,7.750000,10.000000,10.000000,10.000000,7.750000,NaN,0.120000,0.000000,NaN,NaN
max,NaN,NaN,NaN,NaN,1062.000000,8.750000,8.830000,8.670000,8.750000,8.580000,8.750000,10.000000,10.000000,10.000000,10.000000,NaN,0.280000,11.000000,NaN,NaN


# Удаление столбцов

In [4]:
def search_nulles(data):
    nulles = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([nulles, percent], axis=1, keys=['Nulles', 'Percent'])
    return missing_data

search_nulles(data).head(30)

,Nulles,Percent
view_certificate_2,1354,1.000000
Cupping Protocol and Descriptors,1354,1.000000
View Green Analysis Details,1354,1.000000
NA.3,1354,1.000000
Request a Sample,1354,1.000000
Unnamed: 51,1354,1.000000
NA.2,1354,1.000000
view_certificate_1,1352,0.998523
Notes,1349,0.996307
Lot Number,1078,0.796160


Смотрим, сколько в каждом столбце нулевых значений. Так как такие столбцы не несут особой полезности, то в дальнейшем мы не будем добавлять их в таблицу export_table ( это настраивается далее в Talend )

Так как некоторые столбцы полностью заполнены нулевыми значениями, то их можно удалить
Более того, можно удалить столбцы с более 50% пропусков

In [5]:
# data.drop("Cupping Protocol and Descriptors", axis=1, inplace=True)
# data.drop("View Green Analysis Details", axis=1, inplace=True)
# data.drop("NA.3", axis=1, inplace=True)
# data.drop("Request a Sample", axis=1, inplace=True)
# data.drop("Unnamed: 51", axis=1, inplace=True)
# data.drop("NA.2", axis=1, inplace=True)
# data.drop("view_certificate_2", axis=1, inplace=True)
# data.drop("view_certificate_1", axis=1, inplace=True)
# data.drop("Notes", axis=1, inplace=True)
# data.drop("Lot Number", axis=1, inplace=True)

Заметим, что в самом конце таблицы с нулями есть атрибуты с одинаковым количеством нулей. Посмотрим на эти данные. Есть целые строчки с нулевыми значениям. Так как они не несут нам пользы - удалим их

In [6]:
data[data['Farm Name'].isnull() == False][['Owner', 'Owner.1', 'Farm Name', 'Mill', 'Company', 'Producer']].head(50)

,Owner,Owner.1,Farm Name,Mill,Company,Producer
0,metad plc,metad plc,METAD PLC,METAD PLC,METAD Agricultural Developmet plc,METAD PLC
1,metad plc,metad plc,METAD PLC,METAD PLC,METAD Agricultural Developmet plc,METAD PLC
2,Grounds for Health Admin,Grounds for Health Admin,"San Marcos Barrancas ""San Cristobal Cuch",NaN,NaN,NaN
3,Yidnekachew Dabessa,Yidnekachew Dabessa,Yidnekachew Dabessa Coffee Plantation,Wolensu,Yidnekachew Debessa Coffee Plantation,Yidnekachew Dabessa Coffee Plantation
4,metad plc,metad plc,METAD PLC,METAD PLC,METAD Agricultural Developmet plc,METAD PLC
7,Ethiopia Commodity Exchange,Ethiopia Commodity Exchange,Aolme,C.P.W.E,NaN,Bazen Agricultural & Industrial Dev't Plc
8,Ethiopia Commodity Exchange,Ethiopia Commodity Exchange,Aolme,C.P.W.E,NaN,Bazen Agricultural & Industrial Dev't Plc
9,Diamond Enterprise Plc,Diamond Enterprise Plc,Tulla Coffee Farm,Tulla Coffee Farm,DIAMOND ENTERPRISE PLC,Diamond Enterprise Plc
10,Mohammed Lalo,Mohammed Lalo,Fahem Coffee Plantation,NaN,Fahem Coffee Plantation,Fahem Coffee Plantation
11,CQI Q Coffee Sample Representative,CQI Q Coffee Sample Representative,El filo,NaN,Coffee Quality Institute,Alfredo De Jesús López Pérez


In [7]:
data[data['Owner'] != data['Owner.1']][['Owner', 'Owner.1']]

,Owner,Owner.1
219,"CECA, S.A.","CECA,S.A."
364,NaN,NaN
459,NaN,NaN
602,NaN,NaN
848,NaN,NaN
882,NaN,NaN
918,NaN,NaN
921,NaN,NaN
922,NaN,NaN
964,Klem Organics,NaN


In [8]:
# data.drop("Owner.1", axis=1, inplace=True)

Удаляем Owner.1 тк этот столбец фактически дублирует Owner

In [9]:
data[data['NA'] != data['NA.1']][['NA', 'NA.1']]
# data.drop("NA", axis=1, inplace=True)

,NA,NA.1
352,NaN,Sample
379,NaN,Sample
918,NaN,NaN
920,NaN,NaN
922,NaN,NaN
964,NaN,NaN
965,NaN,NaN
966,NaN,NaN
967,NaN,NaN
1089,NaN,NaN


Удаляем NA тк этот столбец фактически дублирует NA.1

# Удаление строк

In [10]:
nulles = {}

def isNaN(num):
    return num != num

def NaN_in_row(data):
    i = 0
    while i < len(data):
        line = data.iloc[i]
        for item in line:
            if isNaN(item):
                if i in nulles.keys():
                    nulles[i] += 1
                else:
                    nulles[i] = 1
        i += 1
    return dict(sorted(nulles.items(), key=operator.itemgetter(1), reverse=True))

print('Посмотрим, сколько есть записей с нулевыми ячейками')
nulles = NaN_in_row(data)
list(nulles.items())[:20]

Посмотрим, сколько есть записей с нулевыми ячейками


[(967, 52),
 (1091, 52),
 (1093, 52),
 (1094, 52),
 (1095, 52),
 (1242, 52),
 (1243, 52),
 (918, 51),
 (922, 51),
 (965, 51),
 (1090, 51),
 (966, 50),
 (1092, 50),
 (1089, 39),
 (964, 36),
 (920, 35),
 (921, 28),
 (1209, 21),
 (334, 20),
 (373, 20)]

In [11]:
d = dict((k, v) for k, v in nulles.items() if v >= 35)
drop = list(d)

data.drop(data.index[drop], inplace=True)

data.reset_index()
print('Удалим строки, в которых больше 34 нулевых значений \nтк вряд ли они несут пользу для нашего анализа, ведь всего 52 столбца')


Удалим строки, в которых больше 34 нулевых значений 
тк вряд ли они несут пользу для нашего анализа, ведь всего 52 столбца


# Заполнение нулей

In [12]:
data[data['Owner'].isnull()][['Owner', 'Farm Name', 'Mill', 'Company', 'Producer']].head(21)

,Owner,Farm Name,Mill,Company,Producer
364,NaN,Los Hicaques,CIGRAH SPS,CIGRAH,Reynerio Zepeda
459,NaN,Los Hicaques,CIGRAH SPS,CIGRAH,Reynerio Zepeda
602,NaN,Supply Chain ECOM CCA S.A.,South Huila,ECOM CCA S.A.,Supply Chain ECOM CCA S.A.
848,NaN,Gran Manzana y el Aguacate,CIGRAH SPS,CIGRAH,"Tomás Sosa, Juan Damaso"
882,NaN,Gran Manzana y el Aguacate,CIGRAH SPS,CIGRAH,"Tomás Sosa, Juan Damaso"
921,NaN,NaN,NaN,NaN,NaN
981,NaN,Los Hicaques,CIGRAH,CIGRAH,Reinerio Zepeda
1050,NaN,Los Hicaques,CIGRAH,CIGRAH,Reinerio Zepeda


In [13]:
data[data['Company'] == 'CIGRAH'][['Owner', 'Farm Name', 'Mill', 'Company', 'Producer', 'Region', 'Altitude', 'ICO Number', 'In-Country Partner']].head(21)

,Owner,Farm Name,Mill,Company,Producer,Region,Altitude,ICO Number,In-Country Partner
364,NaN,Los Hicaques,CIGRAH SPS,CIGRAH,Reynerio Zepeda,Comayagua,1350,13-111-240,Instituto Hondureño del Café
459,NaN,Los Hicaques,CIGRAH SPS,CIGRAH,Reynerio Zepeda,Comayagua,1350,13-111-193,Instituto Hondureño del Café
655,Elsy Reyes,Las Cuchillas,CIGRAH S.A DE C.V,CIGRAH,Nahun Maldonado,Comayagua,1400,13-111-097,Instituto Hondureño del Café
848,NaN,Gran Manzana y el Aguacate,CIGRAH SPS,CIGRAH,"Tomás Sosa, Juan Damaso",Comayagua,1350,13-111-311,Instituto Hondureño del Café
882,NaN,Gran Manzana y el Aguacate,CIGRAH SPS,CIGRAH,"Tomás Sosa, Juan Damaso",Comayagua,1400,13-111-192,Instituto Hondureño del Café
912,Elsy Reyes,LAS CUCHILLAS,CIGRAH,CIGRAH,NAHUN MALDONADO,COMAYAGUA,1400,13-111-083,Instituto Hondureño del Café
981,NaN,Los Hicaques,CIGRAH,CIGRAH,Reinerio Zepeda,Central Region,1450 mals,13-111-037,Instituto Hondureño del Café
1050,NaN,Los Hicaques,CIGRAH,CIGRAH,Reinerio Zepeda,Central Region,1450 mals,13-111-035,Instituto Hondureño del Café


Так как мы хотим использовать Owner как первичный составной ключ, 
то нельзя допустить в нём нулевые значения. Заполняем их уникальным значением владельца в этой компании

In [14]:
owner = data[data['Company'] == 'CIGRAH']['Owner'].unique()[1]
nan_index = data[data['Owner'].isnull()].index
data.loc[nan_index, 'Owner'] = pd.Series(owner, index=nan_index)

In [15]:
nan_index = data[data['Quakers'].isnull()].index
data.loc[nan_index, 'Quakers'] = pd.Series(0, index=nan_index)

# Декомпозиция данных

In [16]:
def split_weight(data):
    for i in tqdm(range(len(data))):
        item = data.iloc[i, 18]
        if isinstance(item, int):
            weigth = item
        elif not isNaN(item):
            words = item.split(' ')
            weigth = int(words[0])
            if len(words) > 1:
                metric = words[1]
                if metric.lower().strip() == 'lbs':
                    weigth = weigth * 0.453592
        data.iloc[i, 18] = round(weigth, 3)
    return data

In [17]:
data.iloc[1, 17:25]

Number of Bags                                       300
Bag Weight                                         60 kg
In-Country Partner    METAD Agricultural Development plc
Harvest Year                                        2014
Grading Date                             April 4th, 2015
Owner.1                                        metad plc
Variety                                            Other
Status                                         Completed
Name: 1, dtype: object

In [18]:
def float_to_int(data):
    for i in tqdm(range(len(data))):
        item = data.iloc[i, 17]
        number = int(item)
        data.iloc[i, 17] = number
    return data

In [19]:
data = split_weight(data)
data['Bag Weight']

100%|██████████| 1338/1338 [00:00<00:00, 2086.34it/s]


0          60
1          60
2           1
3          60
4          60
        ...  
1349        2
1350        2
1351        1
1352    2.268
1353    2.268
Name: Bag Weight, Length: 1338, dtype: object

In [20]:
def split_defects(data, number='one'):
    for i in tqdm(range(len(data))):
        if number == 'one':
            k = 41
        elif number == 'two':
            k = 44
        item = data.iloc[i, k]
        if isNaN(item) is False:
            words = item.split(' ')
            defects = int(words[0])
        data.iloc[i, k] = defects
            
    return data

In [21]:
data = split_defects(data)
data = split_defects(data, 'two')

100%|██████████| 1338/1338 [00:00<00:00, 1977.44it/s]


In [22]:
data['Expiration']

0           April 3rd, 2016
1           April 3rd, 2016
2            May 31st, 2011
3          March 25th, 2016
4           April 3rd, 2016
               ...         
1349     January 18th, 2017
1350     January 18th, 2017
1351    December 23rd, 2015
1352      August 25th, 2015
1353      August 25th, 2015
Name: Expiration, Length: 1338, dtype: object

In [23]:
import datetime

months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
          'May': 5, 'June': 6, 'July': 7, 'August': 8,
          'September': 9, 'October': 10, 'November': 11, 'December': 12}

In [24]:
def string_to_date(data, date='export'):
    if date == 'export':
        k = 46
    elif date == 'grading':
        k = 21
    
    for i in tqdm(range(len(data))):
        item = data.iloc[i, k]
        if not isNaN(item):
            words = item.split(' ')
            if len(words) > 2:
                month = words[0]
                day = words[1][:-3]
                year = words[2]
                data.iloc[i, k] = datetime.date(year=int(year), month=months[month], day=int(day))
    return data

data = string_to_date(data)
data = string_to_date(data, date='grading')
data['Grading Date']

100%|██████████| 1338/1338 [00:00<00:00, 1699.30it/s]


0       2015-04-04
1       2015-04-04
2       2010-05-31
3       2015-03-26
4       2015-04-04
           ...    
1349    2016-01-19
1350    2016-01-19
1351    2014-12-23
1352    2014-08-25
1353    2014-08-25
Name: Grading Date, Length: 1338, dtype: object

In [25]:
import random
import datetime

def add_transact_data(data):
    
    transactions = []
    j = 0
    for i in tqdm(range(len(data))):
        export = data.iloc[i, 46]
        grade = data.iloc[i, 21]
        start_date = grade
        end_date = export
        if not isNaN(start_date):

            time_between_dates = end_date - start_date
            days_between_dates = time_between_dates.days

            random_number_of_days = random.randrange(days_between_dates)
            random_date = start_date + datetime.timedelta(days=random_number_of_days)
            transactions.append(random_date)
        else:
            transactions.append(grade)
        j += 1
    data['Export Date'] = transactions
            
    return data

data = add_transact_data(data)

100%|██████████| 1338/1338 [00:00<00:00, 42688.98it/s]


In [26]:
data['Export Date']

0       2015-06-13
1       2015-11-18
2       2010-07-24
3       2015-06-02
4       2016-01-22
           ...    
1349    2016-08-07
1350    2016-04-10
1351    2015-02-10
1352    2015-07-25
1353    2014-11-05
Name: Export Date, Length: 1338, dtype: object

In [27]:
data.iloc[2, 21]

datetime.date(2010, 5, 31)

In [28]:
data['Number of Bags']

0       300.0
1       300.0
2         5.0
3       320.0
4       300.0
        ...  
1349      1.0
1350      1.0
1351      1.0
1352      1.0
1353      1.0
Name: Number of Bags, Length: 1338, dtype: float64

# Поиск функциональных зависимостей

В этом разделе мы анализировали зависимости: кто чем владеет, что более уникально - мельница или фирма? Обо всём этом здесь. Пояснения зависимостей в основном отчёте

In [29]:
body = data['Certification Body'].unique()
body

array(['METAD Agricultural Development plc',
       'Specialty Coffee Association',
       'Specialty Coffee Institute of Asia',
       'Ethiopia Commodity Exchange', 'Almacafé',
       'Yunnan Coffee Exchange', 'Blossom Valley International',
       'AMECAFE', 'NUCOFFEE', 'Uganda Coffee Development Authority',
       'Instituto Hondureño del Café',
       'Specialty Coffee Association of Costa Rica',
       'Kenya Coffee Traders Association',
       'Africa Fine Coffee Association', 'Asociacion Nacional Del Café',
       'Centro Agroecológico del Café A.C.', 'Salvadoran Coffee Council',
       'Specialty Coffee Association of Indonesia',
       'Brazil Specialty Coffee Association',
       'Asociación Mexicana De Cafés y Cafeterías De Especialidad A.C.',
       'Tanzanian Coffee Board',
       'Central De Organizaciones Productoras De Café y Cacao Del Perú - Central Café & Cacao',
       'Torch Coffee Lab Yunnan', 'Coffee Quality Institute',
       'Asociación de Cafés Especiales de N

In [30]:
partners = data['In-Country Partner'].unique()
for partner in partners:
    if partner not in body:
        print(partner)

Specialty Coffee Ass
nan


In [31]:
nan_index = data[data['In-Country Partner'] != data['Certification Body']].index
data.loc[nan_index, 'In-Country Partner'] = pd.Series('Specialty Coffee Association of Costa Rica', index=nan_index)

Находим только уникальные ICO Numbers, если значение повторяется - не учитываем строку.

In [32]:
ICO = {}

forbidden = [118, 162, 199, 266, 458, 612, 1128]
indexes = []

for i in tqdm(range(len(data))):
    if i not in forbidden:
        item = str(data.iloc[i, 12])
        if not any(map(str.isdigit, item)):
            indexes.append(i)
        else:
            if item not in ICO.keys():
                ICO[item] = 1
            else:
                indexes.append(i)

100%|██████████| 1338/1338 [00:00<00:00, 80176.85it/s]


In [33]:
data.drop(data.index[indexes], inplace=True)
data.reset_index()

,index,quality_score,view_certificate_1,view_certificate_2,Cupping Protocol and Descriptors,View Green Analysis Details,Request a Sample,Species,Owner,Country of Origin,...,Color,Category Two Defects,NA.3,Expiration,Certification Body,Certification Address,Certification Contact,Unnamed: 51,Notes,Export Date
0,0,90.58,NaN,NaN,NaN,NaN,NaN,Arabica,metad plc,Ethiopia,...,Green,0,NaN,2016-04-03,METAD Agricultural Development plc,"BAWA Center, 3rd Floor (Gerji), Addis Ababa, E...","Aman Adinew (Emebet Dinku) - +251-116-292534, ...",NaN,NaN,2015-06-13
1,7,88.67,NaN,NaN,NaN,NaN,NaN,Arabica,Ethiopia Commodity Exchange,Ethiopia,...,NaN,0,NaN,2011-09-02,Ethiopia Commodity Exchange,"Al-Sam Tower 2, 3rd Floor Lideta, Addis Ababa,...",- +251 11 554 7001,NaN,NaN,2010-10-20
2,9,88.25,NaN,NaN,NaN,NaN,NaN,Arabica,Diamond Enterprise Plc,Ethiopia,...,Green,4,NaN,2016-03-29,METAD Agricultural Development plc,"BAWA Center, 3rd Floor (Gerji), Addis Ababa, E...","Aman Adinew (Emebet Dinku) - +251-116-292534, ...",NaN,NaN,2015-04-07
3,17,87.33,NaN,NaN,NaN,NaN,NaN,Arabica,Ethiopia Commodity Exchange,Ethiopia,...,NaN,2,NaN,2011-06-16,Ethiopia Commodity Exchange,"Al-Sam Tower 2, 3rd Floor Lideta, Addis Ababa,...",- +251 11 554 7001,NaN,NaN,2010-11-07
4,19,87.25,NaN,NaN,NaN,NaN,NaN,Arabica,EssenceCoffee,Ethiopia,...,NaN,8,NaN,2016-03-24,Blossom Valley International,No.72 Mo-Fan St. Taiwan Zip: 403,Damon Chen - +886-4-23022323,NaN,NaN,2015-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,1327,83.5,NaN,NaN,NaN,NaN,NaN,Robusta,Nishant Gurjer,India,...,NaN,2,NaN,2018-10-31,Specialty Coffee Association,"117 W 4th St, Suite 300 Santa Ana, CA 92701",Chris Buck - (562) 624-4100,NaN,NaN,2017-11-23
815,1333,82.5,NaN,NaN,NaN,NaN,NaN,Robusta,Nishant Gurjer,India,...,Bluish-Green,0,NaN,2018-10-25,Specialty Coffee Association,"117 W 4th St, Suite 300 Santa Ana, CA 92701",Chris Buck - (562) 624-4100,NaN,NaN,2018-08-20
816,1334,82.5,NaN,NaN,NaN,NaN,NaN,Robusta,Nishant Gurjer,India,...,Green,0,NaN,2017-08-17,Specialty Coffee Association,"117 W 4th St, Suite 300 Santa Ana, CA 92701",Chris Buck - (562) 624-4100,NaN,NaN,2017-04-24
817,1337,81.58,NaN,NaN,NaN,NaN,NaN,Robusta,Nishant Gurjer,India,...,Green,0,NaN,2017-08-23,Specialty Coffee Association,"117 W 4th St, Suite 300 Santa Ana, CA 92701",Chris Buck - (562) 624-4100,NaN,NaN,2017-06-15


In [34]:
set1 = set()
forbidden = [100, 125, 172, 311, 419, 725] # уникальные ICO Numbers, но в формате времени

for i in tqdm(range(data.shape[0])):
    if i not in forbidden:
        for j in range(data.shape[1]):
            item = data.iloc[i, j]
            if isinstance(item, str):
                if len(item) > 49:
                    set1.add(j)
                    
print(f'{len(set1)} столбцов с длиной строки больше 49: {set1}')
print('Данная функция нужна для определения оптимальной длины строковых значений')

100%|██████████| 819/819 [00:00<00:00, 2235.74it/s]

12 столбцов с длиной строки больше 49: {7, 9, 11, 13, 15, 48, 49, 16, 19, 47, 51, 22}
Данная функция нужна для определения оптимальной длины строковых значений


In [35]:
data.iloc[0, 48]

'BAWA Center, 3rd Floor (Gerji), Addis Ababa, Ethiopia'

In [36]:
data['Export Date']

0       2015-06-13
7       2010-10-20
9       2015-04-07
17      2010-11-07
19      2015-09-20
           ...    
1327    2017-11-23
1333    2018-08-20
1334    2017-04-24
1337    2017-06-15
1352    2015-07-25
Name: Export Date, Length: 819, dtype: object

In [37]:
len(data['Producer'].unique())

413

In [38]:
len(data['Farm Name'].unique())

358

In [39]:
len(data['Mill'].unique())

324

In [40]:
len(data['Region'].unique())

300

In [41]:
len(data['Company'].unique())

197

# Выгрузка файлов

Разделяем файл по принципу "раньше/позже 2017 года"

In [42]:
auto_date = data[data['Export Date'] >= datetime.date(2017, 1, 1)]
date = data[data['Export Date'] < datetime.date(2017, 1, 1)]

In [43]:
part = round(len(date) / 2)
part

355

In [44]:
data_for_csv = date[:part]
data_for_excel = date[part:]
data_for_excel_auto = auto_date

In [45]:
data_for_csv.to_csv("data.csv")
data_for_excel.to_excel("data.xlsx")
data_for_excel_auto.to_excel("data_auto.xlsx")